In [13]:
from xml.etree import ElementTree as ET
import urllib.request
from datetime import datetime
import numpy as np
import time
import os

import pandas as pd
import numpy as np

In [4]:
s1= "tormenta.xml"
s2 = "ergo.xml"
t1 = ET.parse(s1)
t2 = ET.parse(s2)

items1 = t1.findall('.//PostItem')
items2 = t2.findall('.//PostItem')


In [9]:
def makeDir():
    paths = ["Photo",
             "Text",
             "Video"]
    try:
        for p in paths:
            os.mkdir(p)
    except OSError:
        print("Creation of the directory %s failed" % p)
    else:
        print("Successfully created the directory %s " % p)


def getType(item):
    """Helper to get the type of a post"""
    thisType = item.find('type').text
    return thisType


def getTerm(item, keyword, isTxt=True):
    """Find tags"""
    try:
        thisTerm = item.find(keyword)
    except AttributeError as e1:
        thisTerm = 'N/A'
        print(e1)
        print("------(QwQ )Not Found------")
    except TypeError as e2:
        thisTerm = 'N/A'
        print(e2)
        print("------(QwQ )Not Found------")
    if thisTerm == None:
        thisTerm = 'N/A'
    if thisTerm == 'N/A':
        return thisTerm
    if isTxt:
        return thisTerm.text
    else:
        return thisTerm


def modifyText(txt):
    txt = txt.replace('<p>', '')
    txt = txt.replace('</p>', '')
    txt = txt.replace('<br />', '\n')
    txt = txt.replace('&nbsp;', ' ')
    txt = txt.replace('&middot;', '·')
    txt = txt.replace('&quot;', '"')
    txt = txt.replace('<a href=', '<')
    txt = txt.replace('</a>', '')
    txt = txt.replace('<li>', '')
    txt = txt.replace('</li>', '')
    return txt

def getContent(item):
    thisCont = getTerm(item, 'content', True)
    thisCont = modifyText(thisCont)
    return thisCont

def getTitle(item):
    thisTitle = getTerm(item, 'titel')
    return modifyText(thisTitle)

def getCaption(item):
    thisCap = getTerm(item, 'caption', True)
    return modifyText(thisCap)


def getTime(item):
    ts = int(getTerm(item, 'publishTime', True))
    tStamp = float(ts/1000)
    tArray = time.localtime(tStamp)
    DateTime = time.strftime("%Y-%m-%d %H:%M:%S", tArray)
    return [ts, DateTime]


def getComments(item):
    cmItem = getTerm(item, 'commentList', False)
    comList = []
    if cmItem != 'N/A':
        com_lst = cmItem.findall('comment')
        for cm in com_lst:
            u_id = getTerm(cm, 'publisherNick', True)
            u_cm = getContent(cm)
            u_time = getTime(cm)
            m = u_id + ': ' + u_cm + '(' + u_time[1] + ')'
            comList.append(m)
    return comList


def getMedia(item, sendTime, tp, file_path):
    if tp == 'Video' or tp == 'Photo':
        if tp == 'Photo':
            links = getTerm(item, 'photoLinks')
            suffix = '.jpg'
            pos1 = links.find('"orign":"')
            if pos1 != -1:
                chopped = links[pos1+9:]
                pos2 = chopped.find('"')
                url = chopped[: pos2]
                print(url)
            else:
                return 'N/A'
        if tp == 'Video':
            links = getTerm(item, 'embed')
            suffix = '.mp4'
            pos1 = links.find('"video_down_url":"')
            if pos1 != -1:
                chopped = links[pos1+18:]
                pos2 = chopped.find('"')
                url = chopped[:pos2]
                print(url)
            else:
                return 'N/A'
        name = file_path + '/' + str(sendTime) + suffix
        print(name)
        urllib.request.urlretrieve(url, name)
        return [name, links]



def getMusic(item):
    all_info = getTerm(item, 'embed')
    pos_mu_1 = all_info.find('"song_name":"')
    if pos_mu_1 != -1:
        chopped_1 = all_info[pos_mu_1+13:]
        pos_mu_2 = chopped_1.find('"')
        mu_name = chopped_1[:pos_mu_2]
    else:
        mu_name = 'N/A'
    pos_url_1 = all_info.find('"listenUrl":"')
    if pos_url_1 != -1:
        chopped_2 = all_info[pos_url_1+13:]
        pos_url_2 = chopped_2.find('"')
        url = chopped_2[:pos_url_2]
    else:
        url = 'N/A'
    return [mu_name, url]

In [10]:

Title = []
Date = []
Type = []
File = []
Link = []
Comments = []
Content = []
Tag = []

def addArchive(t, d, p, f, l, cm, cn, tag):
    Title.append(t)
    Date.append(d)
    Type.append(p)
    File.append(f)
    Link.append(l)
    Comments.append(cm)
    Content.append(cn)
    Tag.append(tag)

def grabMedia(item, tp):
    title = getTitle(item)
    time = getTime(item)
    file, link = getMedia(item, time[0], tp, tp)
    cm = getComments(item)
    cn = getCaption(item)
    tag = getTerm(item, 'tag')
    addArchive(title, time[1], tp, file, link, cm, cn, tag)
    
def grabText(item, tp):
    title = getTitle(item)
    time = getTime(item)
    ts = time[0]
    cn = getContent(item)
    file = 'Text/'+str(ts)+'.txt'
    text_file = open(file, "w")
    n = text_file.write(cn)
    text_file.close()
    cm = getComments(item)
    link = file
    tag = getTerm(item, 'tag')
    addArchive(title, time[1], tp, file, link, cm, cn, tag)

def grabMusic(item, tp):
    title = getTitle(item)
    time = getTime(item)
    cn = getCaption(item)
    file, link = getMusic(item)
    cm = getComments(item)
    tag = getTerm(item, 'tag')
    addArchive(title, time[1], tp, file, link, cm, cn, tag)
    


In [12]:
for item in items1:
    tp = getType(item)
    if tp == 'Video' or tp == 'Photo':
        grabMedia(item, tp)
        print(tp)
    elif tp == 'Text':
        grabText(item, tp)
        print(tp)
    else:
        grabMusic(item, tp)
        print(tp)

http://vodm2lzexwq.vod.126.net/vodm2lzexwq/xk5KBMJk_3029758596_sd.mp4?resId=254486990bfa2cd7aa860229db639341_3029758596_1&sign=DNL8GkIAUSNAvFEDOzPX%2BpYstDagt193EEVoiPWJH9M%3D
Video/1591974305695.mp4


HTTPError: HTTP Error 403: Forbidden

In [14]:
it1 = items1[0]
it1

<Element 'PostItem' at 0x1074c03b8>

In [17]:
ts = str(getTime(it1))
getMedia(it1, ts, 'Video', 'Video')

http://vodm2lzexwq.vod.126.net/vodm2lzexwq/xk5KBMJk_3029758596_sd.mp4?resId=254486990bfa2cd7aa860229db639341_3029758596_1&sign=DNL8GkIAUSNAvFEDOzPX%2BpYstDagt193EEVoiPWJH9M%3D
Video/[1591974305695, '2020-06-12 23:05:05'].mp4


HTTPError: HTTP Error 403: Forbidden